In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import lightgbm as lgbm

from scipy.optimize import minimize

from hyperopt import hp 
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval
from hyperopt.pyll.base import scope

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020

In [5]:
algorithm_name = 'lgbm'

feature_names= ['stacking-layer2']

feature_target_file = feature_dir / f'feature_target.csv'

model_names = []
for feature_name in feature_names:
    model_names.append(f'{algorithm_name}_{feature_name}')
    
stacking_oof_pred_files=[]
for model_name in model_names:
    stacking_oof_pred_files.append( val_dir / f'{model_name}_oof_pred.csv')
    
stacking_test_pred_files=[]
for model_name in model_names:
    stacking_test_pred_files.append( tst_dir / f'{model_name}_test_pred.csv')
    
stacking_submission_files=[]
for model_name in model_names:
    stacking_submission_files.append( sub_dir / f'{model_name}_submission.csv')

# Stacking feature 생성

In [6]:
def load_feature(model_names, number_of_ver=None, kind=None):
    oof_list = []
    test_list = []
    
    if number_of_ver==None or kind==None:
        print('error')
        return None
    
    # 딥러닝 시리즈 4가지 버전
    if kind == 0:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1,number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    # 로지스틱 회귀 6가지 버전
    elif kind == 1:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1, number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))

    # 신경망 기반 불용어 처리 21가지 버전 또는 머신러닝 기반 불용어 처리 18가지 버전
    elif kind == 2:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(2,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 신경만 기반 불용어 처리 X 13가지 버전 또는 머신러닝 기반 불용어 처리 X 18가지 버전
    elif kind == 3:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,2):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
                
    # 첫번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 4:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 두번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 5:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('stacking') != -1:
                for feature in ['stopwords-yes-nn','stopwords-no-nn','stopwords-no-ml', 'stopwords-no-ml'] :
                    oof_list.append(np.loadtxt(val_dir / f'{model}-{feature}_oof_pred.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}-{feature}_test_pred.csv',delimiter=','))
            elif model.find('all') != -1:
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv',delimiter=','))
    
    # 두번째 레이어를 학습하기 위한 PCA 데이터셋 + xgb/lgbm tfidf 데이터셋 가져오기 버전 - 하나씩
    elif kind ==6:
        for model in model_names:
            print(f'load {model}_cv')
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv', delimiter=','))
            test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv', delimiter=','))
            
                        
        
    
    return oof_list, test_list

In [ ]:
model_names = ['mlp_tfidf-pca', 'lr_tfidf-pca', 'cnn_tfidf-pca', 'lgbm_tfidf','lgbm_tfidf-pca']

tmp_oof, tmp_test = load_feature(model_names, -1, 6)
tmp_oof = np.concatenate(tmp_oof, axis=1)
tmp_test = np.concatenate(tmp_test, axis=1)
print(f'shape : {tmp_oof.shape}, {tmp_test.shape}')

In [ ]:
model_names = ['ets_stacking-layer1','gb_stacking-layer1','lgbm_stacking-layer1','mlp_stacking-layer1',
              'rf_stacking-layer1','xgb_stacking-layer1','ada_stacking-layer1',
              'xgb_all','lgbm_all']

all_oof, all_test = load_feature(model_names, -1, 5)
all_oof = np.concatenate(all_oof, axis=1)
all_test = np.concatenate(all_test, axis=1)
print(f'shape : {all_oof.shape}, {all_test.shape}')

In [7]:
all_oof = np.concatenate([all_oof, tmp_oof], axis=1)
all_test = np.concatenate([all_test, tmp_test], axis=1)
print(f'shape : {all_oof.shape}, {all_test.shape}')

load cnn_bow_cv
load cnn_feature_cv
load cnn_hashing_cv
load cnn_tfidf_cv
load lr_bow_cv
load lr_hashing_cv
load lr_tfidf_cv
load lstm_feature_cv
load mlp_bow_cv
load mlp_feature_cv
load mlp_hashing_cv
load mlp_tfidf_cv
load transformer_feature_cv
load transformer_v2_feature_cv
nn_yes shape : (54879, 370), (19617, 370)


In [8]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['index',target_col]).values.flatten()
y.shape

(54879,)

# 하이퍼파라미터 튜닝




In [9]:
# LightGBM parameters
lgb_reg_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
    'metric': 'multi_logloss',
    'objective': 'multiclass',
    'num_class': 5,
    'random_state': 2020,
    'verbosity': 0,

    'boosting': 'gbdt',
    'bagging_freq': scope.int(hp.uniform('bagging_freq',1,10)),
    'max_depth': hp.choice('max_depth',np.arange(5,30,1,dtype=int)),
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'num_leaves': scope.int(hp.quniform('num_leaves', 30, 150, 1)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': scope.int(hp.quniform('subsample_for_bin', 20000, 300000, 20000)),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': scope.int(hp.qloguniform('min_data_in_leaf', 0, 6, 1)),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)]),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
}
lgb_fit_params = {
    'eval_metric': 'multi_logloss',
    'early_stopping_rounds': 30,
    'verbose': True,
}

lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred, labels=[0,1,2]))

In [10]:
class HPOpt(object):

    def __init__(self, X_train, X_val, y_train, y_val):
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result

    def lgb_clf(self, para):
        clf = lgbm.LGBMClassifier(**para['reg_params'])
        return self.train_reg(clf, para)
    
    def train_reg(self, clf, para):
        clf.fit(self.X_train, self.y_train,
                eval_set=[(self.X_train, self.y_train), (self.X_val, self.y_val)],
                **para['fit_params'])
        
        loss = clf.best_score_['valid_1']['multi_logloss']
        return {'loss': loss, 'status': STATUS_OK, 'model': clf}

In [11]:
X_train, X_val, y_train, y_val = train_test_split(all_oof, y, test_size= 0.25, random_state=2020)

obj = HPOpt(X_train, X_val, y_train, y_val)

trials=Trials()
best = obj.process(fn_name='lgb_clf', space=lgb_para, trials=trials, algo=tpe.suggest, max_evals=30)

[LightGBM] [Warning]                                 
bagging_fraction is set=0.5927533847543784, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5927533847543784
[LightGBM] [Warning]                                 
lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning]                                 
min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning]                                 
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning]                                 
boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning]                                 
feature_fraction is set=0.780179196497695, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.780179196497695
[LightGBM] [Warning]                                 
lambda_l2 is

Training until validation scores don't improve for 30 rounds                   
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[2]	training's multi_logloss: 1.52801	valid_1's multi_logloss: 1.52986         
[LightGBM] [Warning]                    

No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[20]	training's multi_logloss: 1.04884	valid_1's multi_logloss: 1.06569        
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[39]	training's multi_logloss: 0.779297	valid_1's multi_logloss: 0.810518      
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, be

[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[57]	training's multi_logloss: 0.623974	valid_1's multi_logloss: 0.667685      
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, be

...

No further splits with positive gain, best gain: -inf                          
[922]	training's multi_logloss: 0.308529	valid_1's multi_logloss: 0.403936     
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[923]	training's multi_logloss: 0.30844	

[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[941]	training's multi_logloss: 0.30678	valid_1's multi_logloss: 0.40368       
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, be

[959]	training's multi_logloss: 0.305122	valid_1's multi_logloss: 0.403551     
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[960]	training's multi_logloss: 0.305037	valid_1's multi_logloss: 0.403536     
[LightGBM] [Warning]                    

No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[978]	training's multi_logloss: 0.303448	valid_1's multi_logloss: 0.403431     
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[LightGBM] [Warning]                                                           
No further splits with positive gain, best gain: -inf                          
[997]	training's multi_logloss: 0.301754	valid_1's multi_logloss: 0.403365     
[LightGBM] [Warning]                                                           
No further splits with positive gain, be

In [12]:
best

{'bagging_fraction': 0.6086947651883814,
 'bagging_freq': 8.169979235096044,
 'class_weight': 0,
 'feature_fraction': 0.9534425277699308,
 'lambda_l1': 0,
 'lambda_l2': 0,
 'learning_rate': 0.03262235114708719,
 'max_depth': 20,
 'min_child_weight': 142.7460918279221,
 'min_data_in_leaf': 4.0,
 'num_leaves': 41.0,
 'subsample_for_bin': 240000.0}

In [13]:
hyperparams = space_eval(lgb_reg_params, best)
hyperparams

{'bagging_fraction': 0.6086947651883814,
 'bagging_freq': 8,
 'boosting': 'gbdt',
 'class_weight': None,
 'feature_fraction': 0.9534425277699308,
 'lambda_l1': 0,
 'lambda_l2': 0,
 'learning_rate': 0.03262235114708719,
 'max_depth': 25,
 'metric': 'multi_logloss',
 'min_child_weight': 142.7460918279221,
 'min_data_in_leaf': 4,
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_class': 3,
 'num_leaves': 41,
 'objective': 'multiclass',
 'random_state': 2020,
 'subsample_for_bin': 240000,
 'verbosity': 0}

In [14]:
trials.best_trial['result']['model']

LGBMClassifier(bagging_fraction=0.6086947651883814, bagging_freq=8,
               boosting='gbdt', feature_fraction=0.9534425277699308,
               lambda_l1=0, lambda_l2=0, learning_rate=0.03262235114708719,
               max_depth=25, metric='multi_logloss',
               min_child_weight=142.7460918279221, min_data_in_leaf=4,
               n_estimators=1000, num_class=3, num_leaves=41,
               objective='multiclass', random_state=2020,
               subsample_for_bin=240000, verbosity=0)